# Integrating imaging and sequencing

In the previous step, the flow-cell barcodes were mapped to spatial coordinates, and the transcriptomic reads were processed and mapped in tissue space with `spacemake`. 

With those steps, you obtained `h5ad` objects where each cell is a [hexbin](https://h3geo.org/docs/comparisons/hexbin/), containing around ~400 of the 0.6µm barcoded spots, and can be used already for analysis.

For more accurate single-cell representations from the spatial data, we perform image-informed segmentation. This involves three major steps:

1. Pairwise alignment between the imaging and spatial transcriptomics modality
2. Segmentation of the imaging modality
3. Assign transcripts into individual (segmented) cells.

We will illustrate how to do this in a semiautomatic manner: that is, running the coarse alignment in an automatic fashion, and the fine alignment (to fiducial marks) via a Graphical User Interface (GUI), in an interactive manner. This is typically a quick process (~5 minutes per sample of 12 tiles).

We start again at the `spacemake` root folder:

```bash
WORKDIR="~/openst_demo"
cd $WORKDIR/spacemake
```

## Stitching of spatial coordinates

If you ran `spacemake` with a run mode that meshes the data, and no run mode that does not mesh the data, you will need to spatially stitch the single spots data into a single file before pairwise alignment and segmentation. That is, the typical output for `spacemake` consists of several `h5ad` files, one per `tile`, and we require a single file for the sample that contains all tiles in their correct spatial offsets. We do this automatically by running the following:

```bash
for sample in mLN_S{2..4}; do
    openst from_spacemake \
        --project-id mLN \
        --sample-id "$sample" \
        spatial_stitch \
        --tile-coordinates $WORKDIR/spacemake/puck_data/openst_coordinate_system.csv
done
```

There will be a new `h5ad` file that contains all spatial coordinates at the single spot level (0.6 micron resolution), instead of the arbitrary meshing to pseudocells. This is what we require for taking advantage of image-based cell segmentation.

## Merging the modalities (image and ST)

We assume that the image was stitched automatically with the `openst` code above. Then, all files will be at the expected locations and you can run the following:

```bash
for sample in mLN_S{2..4}; do
    openst from_spacemake \
        --project-id mLN \
        --sample-id "$sample" \
        merge_modalities \
        --image-in "${WORKDIR}"/raw_data/images/"${sample}".tif
done
```

This will store the image data into the `h5ad` object, for pairwise alignment and cell segmentation.

## Pairwise alignment

!!! info
    Pairwise alignment should be done in a per-section basis, since manual, qualitative validation of the results is recommended. 

    Therefore, make sure to repeat these steps for the sections `mLN_S2`, `mLN_S3` and `mLN_S4`.

You can run automatic pairwise alignment of the two modalities by running this command:

```bash
openst from_spacemake \
    --project-id mLN \
    --sample-id mLN_S2 \
    pairwise_aligner \
    --device cuda
```

Pairwise alignment happens in two steps, coarse and fine. At the coarse level, low-resolution image and ST data are used, then once the rotation, flip level and offset is more or less set, high-resolution image and ST are used for finding the most accurate alignment, relying on small features from the tissue and the concentric circles (fiducial marks) that are visible at both image and the ST modalities.

It is good practice to check the quality of the alignment before proceeding with the rest of the pipeline. We provide a GUI tool to do this:

```bash
openst from_spacemake \
    --project-id mLN \
    --sample-id mLN_S2 \
    manual_pairwise_aligner
```

You can find more instructions for how to use this tool in this [YouTube video](https://www.youtube.com/watch?v=Fjh4huosVPY). 

If the alignment does not look optimal (e.g., fiducial circles visible at each tile, across both modalities, do not look perfectly aligned), you can use this tool for manual refinement of the alignment. 

This tool helps to store user-selected keypoints as a standard `json` file, that can be used to compute the transformation matrices that align the ST data into the imaging modality. When such a `json` file is available, you can compute the optimal transformation with the following tool:

```bash
openst from_spacemake \
      --project-id mLN \
      --sample-id mLN_S2 \
      apply_transform \
      --keypoints-in keypoints.json \
      --spatial-key-in obsm/spatial_pairwise_aligned_coarse \ # or obsm/spatial_pairwise_aligned_fine
      --spatial-key-out obsm/spatial_manual_fine \
      --per-tile
```

## Cell segmentation

Once the ST and imaging modalities have been aligned, you can segment the images into single cells/nuclei, and then aggregate the spot locations into individual cells for subsequent analysis.

```bash
for sample in mLN_S{2..4}; do
     openst from_spacemake \
          --project-id mLN \
          --sample-id "$sample" \
          segment \
          --model HE_cellpose_rajewsky \
          --device cuda # ignore if no CUDA-compatible GPU is available 
done
```

You can assess the segmentation masks with the `openst preview` tool, based on the `napari` image viewer. You can do this section by section, e.g., for section `mLN_S2`:

```bash
openst from_spacemake \
     --project-id mLN \
     --sample-id mLN_S2 \
     preview \
     --image-keys uns/spatial/staining_image uns/spatial/staining_image_mask
```

## Single-cell quantification

Then, you can create a single file containing the transcriptomic information aggregated into (segmented) single-cells.

```bash
for sample in mLN_S{2..4}; do
     openst from_spacemake \
          --project-id mLN \
          --sample-id "$sample" \
          transcript_assign \
          --spatial-key obsm/spatial_manual_fine \ # or obsm/spatial_pairwise_fine, if the automated results are satisfying
          --mask-in uns/spatial/staining_image_mask
done
```

## Next steps

After running all steps above, you will have a single `h5ad` file that contains the cell-by-gene matrix, using the segmented cells and not the pseudocells that were generated by `spacemake` (by default, a regular grid of 7-micron side hexagons).

This file can be found at (for `mLN_S2`):

`$WORKDIR/spacemake/projects/mLN/processed_data/mLN_S2/multimodal/stitched_segmented.h5ad`

This single object contains transcriptomic data per segmented cell, the imaging and segmentation information, and can be used for downstream analysis.